In [1]:
# setting device on GPU if available, else CPU
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print()

# Additional Info when using cuda
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    print("Memory Usage:")
    print("Allocated:", round(torch.cuda.memory_allocated(0) / 1024**3, 1), "GB")
    print("Cached:   ", round(torch.cuda.memory_reserved(0) / 1024**3, 1), "GB")

Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from functools import partial
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

## test rendering

In [4]:
def findAllFile(base):
    """
    Recursively find all files in the specified directory.

    Args:
        base (str): The base directory to start the search.

    Returns:
        list: A list of file paths found in the directory and its subdirectories.
    """
    file_path = []
    for root, ds, fs in os.walk(base, followlinks=True):
        for f in fs:
            fullname = os.path.join(root, f)
            file_path.append(fullname)
    return file_path

In [5]:
from utils.motion_processing.hml_process import recover_from_ric, recover_root_rot_pos
import utils.vis_utils.plot_3d_global as plot_3d
import matplotlib.pyplot as plt

In [6]:
from core.param_dataclasses import pattern_providers
from core.datasets.multimodal_dataset import MotionIndicesAudioTextDataset, load_dataset_gen, simple_collate
from core.models.utils import instantiate_from_config, get_obj_from_str
from core import MotionRep, AudioRep, TextRep
from core.datasets.conditioner import ConditionProvider,ConditionFuser

from configs.config import cfg, get_cfg_defaults
from configs.config_streaming import get_cfg_defaults as strm_get_cfg_defaults


/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
len(findAllFile("/srv/hays-lab/scratch/sanisetty3/motionx/indices/tmp/beat"))

10355

In [27]:
choreo = findAllFile("/srv/hays-lab/scratch/sanisetty3/motionx/indices/body_gpvc/choreomaster")

In [28]:
beat[0]

'/srv/hays-lab/scratch/sanisetty3/motionx/indices/body_gpvc/beat/7_sophie_0_26_26.npy'

In [29]:
np.load(beat[0]).shape

(1, 553)

In [30]:
for pth in tqdm(beat):
    name = os.path.basename(pth).split(".")[0]
    mot = np.load(pth)
    for i, b in enumerate(range(0 , mot.shape[-1] , 150 )):
        new_name= name + f"_clip{i}.npy"
        new_mot= mot[:,b:b+150]
        np.save(f"/srv/hays-lab/scratch/sanisetty3/motionx/indices/tmp/beat/{new_name}", new_mot)

100%|██████████████████████████████████████████████████████████████████████████████████████| 1620/1620 [08:13<00:00,  3.28it/s]


In [25]:
44/7.5

5.866666666666666

In [7]:
gen_cfg = strm_get_cfg_defaults()
gen_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_streaming/motion_streaming.yaml")
gen_cfg.freeze()

In [8]:

body_cfg = get_cfg_defaults()
body_cfg.merge_from_file(gen_cfg.vqvae.body_config)
# left_cfg = get_cfg_defaults()
# left_cfg.merge_from_file(gen_cfg.vqvae.left_hand_config)
# right_cfg = get_cfg_defaults()
# right_cfg.merge_from_file(gen_cfg.vqvae.right_hand_config)

In [ ]:
from core.models.resnetVQ.vqvae import HumanVQVAE


In [10]:
# left_hand_model = HumanVQVAE(left_cfg.vqvae).to(device).eval()
# left_hand_model.load(os.path.join(left_cfg.output_dir, "vqvae_motion.pt"))

# right_hand_model = HumanVQVAE(right_cfg.vqvae).to(device).eval()
# right_hand_model.load(os.path.join(right_cfg.output_dir, "vqvae_motion.pt"))

body_model = HumanVQVAE(body_cfg.vqvae).to(device).eval()
body_model.load(os.path.join(body_cfg.output_dir, "vqvae_motion.pt"))



### VQVAE

In [8]:
from core.datasets.vq_dataset import VQSMPLXMotionDataset
from core.datasets.vq_dataset import load_dataset as load_dataset_vq
from core.datasets.vq_dataset import simple_collate as simple_collate_vq


In [9]:
def get_decoded(model , motion):
    encs = []
    inds = []
    for i in range(0 , motion.shape[1] , 120 ):
        if i + 240 >= motion.shape[1]:
            enc_b = model(motion[: , i:, :].to(device))
            encs.append(enc_b.decoded_motion)
            inds.append(enc_b.indices)
            break
        else:
            enc_b = model(motion[: , i:i + 120, :].to(device))
            encs.append(enc_b.decoded_motion)
            inds.append(enc_b.indices)
    return torch.cat(encs , 1), torch.cat(inds , -1)

In [10]:
dataset_names_default = [
    "animation",
    "humanml",
    "perform",
    "GRAB",
    "idea400",
    "humman",
    "beat",
    "game_motion",
    "music",
    "aist",
    "fitness",
    "moyo",
    "choreomaster",
    "dance",
    "kungfu",
    "EgoBody",
    "HAA500",
]

In [11]:
body_cfg = get_cfg_defaults()
body_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/vqvae/vqvae_body_gpvc/vqvae_body_gpvc.yaml")

In [12]:
from core.models.resnetVQ.vqvae import HumanVQVAE2

body_model = HumanVQVAE2(body_cfg.vqvae).to(device).eval()
body_model.load(os.path.join(body_cfg.output_dir, "vqvae_motion.pt"))



Sync is turned on False
loaded model with  0.03015906736254692 tensor([110000.], device='cuda:0') steps


In [13]:
# body_cfg.dataset.hml_rep = "gprv"
body_cfg.dataset.window_size = -1

In [18]:
test_ds, _, _ = load_dataset_vq(
            dataset_names=dataset_names_default,
            dataset_args=body_cfg.dataset,
            split="test",
        )

Total number of motions animation: 17
Total number of motions humanml: 3546
Total number of motions perform: 24
Total number of motions GRAB: 67
Total number of motions idea400: 626
Total number of motions humman: 38
Total number of motions beat: 162
Total number of motions game_motion: 511
Total number of motions music: 179
Total number of motions aist: 74
Total number of motions fitness: 837
Total number of motions moyo: 9
Total number of motions choreomaster: 2
Total number of motions dance: 9
Total number of motions kungfu: 52
Total number of motions EgoBody: 49
Total number of motions HAA500: 262


In [19]:
condition_provider2 = ConditionProvider(
            motion_rep=MotionRep("body"),
            motion_padding="max_length",

        )

/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [20]:
dl = torch.utils.data.DataLoader(
            test_ds,
            batch_size=1,
            sampler=None,
            shuffle=False,
            collate_fn=partial(simple_collate_vq , conditioner = condition_provider2),
            
        )

In [57]:
for batch in tqdm(dl):
    break


  0%|                                                                                                  | 0/312 [00:00<?, ?it/s]


In [21]:
errors = []
for batch in tqdm(dl):
    sve = os.path.join("/srv/hays-lab/scratch/sanisetty3/motionx/indices/body_gpvc" , batch["names"][0] )
    if os.path.exists(sve):
        continue
    os.makedirs(os.path.dirname(sve) , exist_ok=True)
    try:
        if "g" in body_cfg.dataset.hml_rep:
            l = list(range(0, batch["motion"][0].shape[-1]))
            ohprvc = l[:1] + l[3:]
            gt_motion = batch["motion"][0][..., ohprvc]
        enc_b, inb = get_decoded(body_model , gt_motion.to(device))
        np.save(sve , inb.cpu().numpy())
    except:
        errors.append(batch["names"][0])
    

100%|██████████████████████████████████████████████████████████████████████████████████████| 6464/6464 [11:57<00:00,  9.01it/s]


In [22]:
len(errors)

9

## Translation

In [4]:
from core.datasets.vq_dataset import VQSMPLXMotionDataset
from core.datasets.vq_dataset import load_dataset as load_dataset_vq
from core.datasets.vq_dataset import simple_collate as simple_collate_vq

/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dset = VQSMPLXMotionDataset("animation" , "/srv/hays-lab/scratch/sanisetty3/motionx" ,hml_rep = "gc" , window_size=120, motion_rep = "body", split = "train" , fps = 30  )


Total number of motions animation: 120


In [6]:
dset[0]

{'name': 'animation/subset_0000/Ways_To_Catch_Coin_Toss.npy',
 'motion': <core.datasets.motion_class.Motion at 0x7f7609fcfa90>}